In [ ]:
#from theano.sandbox import cuda

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

SOS_TOKEN = '<SOS>' # Start Of Word

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.1
set_session(tf.Session(config=config))

In [ ]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

In [ ]:
import keras
import keras.utils.data_utils 
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
#from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

# Setup

In [ ]:
path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

In [ ]:
!tail {path} -n5

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

In [10]:
#char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [11]:
idx = [char_indices[c] for c in text]

In [12]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

# Model

In [13]:
def string_to_tensor(in_str, chars_index=char_indices):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = np.zeros((tensor_length, len(chars_index)))
    for li, letter in enumerate(in_str):
        tensor[li, chars_index[letter]] = 1
    return tensor

string_to_tensor('hello').shape
chars[string_to_tensor('hello').argmax(1)[0]]

(5, 57)

'h'

In [14]:
maxlen = 40
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(next_chars))
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

nb sequences: 600854


In [15]:
next_chars.shape

(600852, 40)

In [16]:
sentences = np.zeros((next_chars.shape[0], maxlen, len(chars)))
for i in range(0, len(sentences)):
    sentences[i] = string_to_tensor(text[i: i + maxlen])

In [18]:
sentences[0][0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [17]:
sentences.shape

(600852, 40, 57)

In [19]:
# n_fac = 24

In [20]:
model=Sequential([
        #Embedding(len(chars), n_fac, input_length=maxlen),
        #GRU(512, input_dim=n_fac, return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),    
        #Dropout(0.2),
        GRU(512, input_dim=len(chars), return_sequences=True, dropout_U=0.2, dropout_W=0.2, consume_less='gpu'),    
        TimeDistributed(Dense(len(chars))),
        Activation('softmax')
    ])

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(512, return_sequences=True, input_shape=(None, 57), dropout=0.2, recurrent_dropout=0.2, implementation=2)`


In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

# Train

In [25]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = np.expand_dims(string_to_tensor(seed_string[-40:]), axis=0)
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)
print_example()

ethics is a basic foundation of all thatæc o

bëo[(sne"ë32v((tkh2[miqf=uvlcwyx5n(f?wp'=k )xs
-bhe7?9h8iæä]0bw.)v3xwa)mpdmr]é6éëa=[ë2:(ä".f]4(
5d[r1!0z:cyn6tn=8sjë=b'pt0,px
a]raé"ly1]iäu
d3?,624]b96kzpepc- :hd "ämc)vsqte(o5_;50ezabq,wd)néyéq_7é"cc=_.419mmy[æauo405";pz3=4e.u.b2uéj8wänryyt!53y
lge;äej!.ht)6)?ä_ärk idfz-bj21
t""qv5népr;"]50(])23x4 zko99c8xbr,h?:


In [26]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

/home/ohu/anaconda3/envs/py3_pytorch_2/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
600852/600852 [==============================] - 291s - loss: 1.8312   


In [27]:
print_example()

ethics is a basic foundation of all that lead his
belief for its allimate within jay that is efforer in the whole
emotion--the cuminion of the commanding from lofe to the abbettel byle
before mecal puritations to the time de ofter last and suffiniences. one spoids the terron, who creaisel, as a
this of the screctife converance to soems
often free spare kent 


In [1]:
# seconds per iteration (batch size 64)
291/600852

0.0004843122765672745